# Image Encoding

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from google.colab import files
from PIL import Image

In [ ]:
modal = models.resnet50(pretrained=True)
modal.eval()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 168MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
uploaded = files.upload()
filename = list(uploaded.keys())[0]

img = Image.open(filename).convert("RGB")
input_tensor = preprocess(img).unsqueeze(0)


Saving angry-cat.jpg to angry-cat.jpg


In [ ]:
feature_extractor = torch.nn.Sequential(*list(modal.children())[:-1])

with torch.no_grad():
    features = feature_extractor(input_tensor)

features = features.squeeze().numpy()
print(features.shape)

(2048,)


# Text Encoding with BERT

In [ ]:
from transformers import AutoTokenizer, AutoModel

bert_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModel.from_pretrained(bert_model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
text = "I'm not in danger! I'm the danger!"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = bert_model(**inputs)
    text_features_bert = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

print("BERT text embedding shape:", text_features_bert.shape)

BERT text embedding shape: (768,)


# Text Encoding with CLIP --- for multimodal alignment with image embeddings

In [ ]:
from transformers import CLIPTokenizer, CLIPTextModel

clip_model_name = "openai/clip-vit-base-patch32"
clip_tokenizer = CLIPTokenizer.from_pretrained(clip_model_name)
clip_text_model = CLIPTextModel.from_pretrained(clip_model_name)

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [ ]:
text = "I'm not in danger! I'm the danger!"
clip_inputs = clip_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    clip_outputs = clip_text_model(**clip_inputs)
    text_features_clip = clip_outputs.last_hidden_state[:, 0, :].squeeze().numpy()

print("CLIP text embedding: ", text_features_clip)
print("CLIP text embedding shape: ", text_features_clip.shape)

CLIP text embedding:  [ 3.39285910e-01  1.16460182e-01  1.01951092e-01  3.09531894e-02
 -2.63870239e+01  1.32117614e-01  1.12613298e-01  1.82474881e-01
  2.64280885e-01  3.22633564e-01  3.46613646e-01  1.09220058e-01
  7.12012313e-03  7.05018290e-04  7.48956054e-02 -3.23447526e-01
  3.45323712e-01  1.18573233e-01  5.53426333e-02  1.76489726e-01
 -1.57664344e-01  3.62499893e-01  4.75042462e-01  1.57557771e-01
  2.32348040e-01 -1.22020226e-02  8.77761692e-02  2.99425483e-01
  8.58490169e-02  4.13953722e-01  4.85188812e-01  8.95347074e-02
  7.64524862e-02 -1.04537554e-01  2.70532399e-01  1.70507029e-01
  2.98013657e-01  1.17371053e-01  2.73247838e-01  4.18955564e-01
 -7.89983720e-02  2.45921299e-01  2.79684216e-02 -5.59642538e-02
  3.37330788e-01 -3.59432586e-02  5.78767359e-02 -4.12365533e-02
 -1.86849236e-01 -9.08554941e-02  3.06749821e-01  9.11944732e-02
  6.98397756e-01 -1.74903005e-01 -8.34562480e-02  5.43052197e-01
  1.86807826e-01  2.94866841e-02  1.38327688e-01  1.61817342e-01
  1

# Multimodal Embedding with CLIP --> Image + text Similarity


In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
from torch.nn.functional import cosine_similarity

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

In [ ]:
url = "https://images.unsplash.com/photo-1592194996308-7b43878e84a6"
image = Image.open(requests.get(url, stream=True).raw)


In [ ]:
texts = ["a photo of a cat", "a photo of a dog", "a photo of a car"]

inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)


In [ ]:
with torch.no_grad():
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds


In [ ]:
# Normalize for cosine similarity
image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)
text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)


In [ ]:
#Compare similarity
similarities = cosine_similarity(image_embeds, text_embeds)
print("Cosine similarities:", similarities.tolist())

Cosine similarities: [0.26390841603279114, 0.22244815528392792, 0.1778745949268341]
